In [ ]:
import os
import sys
import unittest
from math import pi
import numpy as np
import logging
from itertools import permutations
from ddt import ddt, data
from numpy.testing import assert_allclose
import contextlib

from qiskit import transpile
from qiskit.exceptions import QiskitError
from qiskit.circuit import QuantumCircuit, QuantumRegister
from qiskit.providers.basicaer import QasmSimulatorPy
from qiskit.quantum_info.random import random_unitary, random_statevector, random_pauli
from qiskit.quantum_info.states import Statevector
from qiskit.circuit.library import QuantumVolume
from qiskit.providers.aer import AerSimulator
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp
from qiskit.quantum_info.operators.predicates import matrix_equal
from qiskit.visualization.state_visualization import numbers_to_latex_terms, state_to_latex
from qiskit.circuit.library import QFT, HGate
from test.terra import common
from qiskit_aer.aererror import AerError
from qiskit_aer.quantum_info.states import AerStatevector

In [ ]:
def assertEqual(a, b):
    assert a == b

def assertTrue(a):
    assertEqual(a, True)

def assertFalse(a):
    assertEqual(a, False)

def assertIsNone(a):
    assertTrue(a is None)

def assertIn(a, b):
    assertTrue(a in b)

def assertRaises(exception, func, *args, **kwds):
    try:
        func(*args, **kwds)
    except exception:
        pass

def assertAlmostEqual(a, b):
    assertTrue(np.isclose(a, b))

# https://stackoverflow.com/questions/23549419/assert-that-two-dictionaries-are-almost-equal
def _assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    """
    If dictionary value is a number, then check that the numbers are almost equal, otherwise check if values are exactly equal
    Note: does not currently try converting strings to digits and comparing them. Does not care about ordering of keys in dictionaries
    Just returns true or false
    """
    if len(dict1) != len(dict2):
        return False
    # Loop through each item in the first dict and compare it to the second dict
    for key, item in dict1.items():
        # If it is a nested dictionary, need to call the function again
        if isinstance(item, dict):
            # If the nested dictionaries are not almost equal, return False
            if not _assertDictAlmostEqual(dict1[key], dict2[key], rel_tol=rel_tol):
                return False
        # If it's not a dictionary, then continue comparing
        # Put in else statement or else the nested dictionary will get compared twice and
        # On the second time will check for exactly equal and will fail
        else:
            # If the value is a number, check if they are approximately equal
            if isinstance(item, Number):
                # if not abs(dict1[key] - dict2[key]) <= rel_tol:
                # https://stackoverflow.com/questions/5595425/what-is-the-best-way-to-compare-floats-for-almost-equality-in-python
                if hasattr(item, 'imag'):
                    if not math.isclose(dict1[key].real, dict2[key].real, rel_tol=rel_tol) or not math.isclose(dict1[key].imag, dict2[key].imag, rel_tol=rel_tol):
                        return False
                else:
                    if not math.isclose(dict1[key], dict2[key], rel_tol=rel_tol):
                        return False
            else:
                if not (dict1[key] == dict2[key]):
                    return False
    return True

def assertDictAlmostEqual(dict1, dict2, rel_tol=1e-8):
    assertTrue(_assertDictAlmostEqual(dict1, dict2, rel_tol))

@contextlib.contextmanager
def subTest(**kwds):
    try:
        yield
    except Exception as e:
        msg = kwds['msg'] if 'msg' in kwds else ''
        print(e, msg, file=sys.stderr)

In [ ]:
"""Test generation of Aer's StateVector with QV """
circ = QuantumVolume(5, seed=1111)
state = AerStatevector(circ)
expected = Statevector(circ)

for e, s in zip(expected, state):
    assertAlmostEqual(e, s)

In [ ]:
"""Test method and device properties"""
circ = QuantumVolume(5, seed=1111)
state1 = AerStatevector(circ)

assertEqual('statevector', state1.metadata()['method'])
assertEqual('CPU', state1.metadata()['device'])

state2 = AerStatevector(circ, method='matrix_product_state')
assertEqual('matrix_product_state', state2.metadata()['method'])
assertEqual('CPU', state2.metadata()['device'])

assertEqual(state1, state2)